# Laboratorio 7 Deep
## Atención y transformadores
### Abner Iván Garcia Alegria - 21285
### Universidad del valle de Guatemala
### Javier Fong
### Deep Learning 2024


In [3]:
import torch #libreia de pytorch
import torch.nn as nn #modulo de redes neuronales
import math #modulo de matematicas

### Explicación de la clase LayerNormalization
Esta clase es para la normalizacion de capas para un modelo de una red neuronal transformers, la normalizacion en capas nos ayuda a ajustar los valores de entrada para tener una media cercana un poco a 0 y la desviacion cercana a 1. Ahora para lo teorico Utilizamos normalizacion en la funcion forward para calcular la media y la desviacion estandar de x como vimos en la clase calculamos los valores de x en la ultima dimension y calculamos la desviacion de esos valores de x.

In [4]:
# Clase para la normalizacion de capas para redes neuronales
class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps:float=10**-6) -> None: #Constructor de la clase LayerNormalization
        super().__init__() #Inicializa la clase padre
        self.eps = eps #epsilon es una constante que se le asigna a la clase
        self.alpha = nn.Parameter(torch.ones(features)) # alpha es un parametro que se le asigna a la clase
        self.bias = nn.Parameter(torch.zeros(features)) # bias es un parametro que se le asigna a la clase

    def forward(self, x): # funcion forward de la clase LayerNormalization
        mean = x.mean(dim = -1, keepdim = True) # se calcula la media de x en la ultima dimension
        std = x.std(dim = -1, keepdim = True)  # se calcula la desviacion estandar de x en la ultima dimension
        return self.alpha * (x - mean) / (std + self.eps) + self.bias # se retorna el valor normalizado de x

### Explicación de la clase FeedForwardBlock
Esta clase lo que hace es que procesa los datos de manera eficiente mediante un modelo en una red neuronal transformers lo cual crea 2 como capas lineales lo cual complementado con lo teorico estas capas toma la entrada de un vector de tamaño n y como que lo proyecta en un espacio de mayor tamaño, en cambio la segunda capa es como lo contrario al primero ya que este hace lo contrario es decir reduce las dimensiones.

In [5]:
# Clase para la atencion multi-cabeza de las redes neuronales
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None: #Constructor de la clase FeedForwardBlock
        super().__init__() #Inicializa la clase padre de FeedForwardBlock
        self.linear_1 = nn.Linear(d_model, d_ff)  # se crea una capa lineal con d_model entradas y d_ff salidas
        self.dropout = nn.Dropout(dropout) # se crea una capa de dropout con el valor de dropout asignado a la clase FeedForwardBlock
        self.linear_2 = nn.Linear(d_ff, d_model)  # se crea una capa lineal con d_ff entradas y d_model salidas 

    def forward(self, x): # funcion forward de la clase FeedForwardBlock 
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x)))) # se retorna el valor de la segunda capa lineal despues de aplicarle la funcion de activacion relu y dropout

### Explicación de la clase InputEmbeddings
Esta clase como bien su nombre lo dice es como insertar posiciones de entrada que son como palabras numericas el cual genera vectores podriamos decir para que un modelo pueda entender estas palabras dentro de un contexto es como decir que tenemos comentarios de personas random y queremos saber que sentimientos da estos comentarios el cual esta clase puede hacer eso ya que se encarga de ver estas palabras y determinar que sentimientos puede causar dicho comentario.

In [6]:
# Clases para insertar la posicion de las palabras en la red neuronal
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None: #Constructor de la clase InputEmbeddings 
        super().__init__() #Inicializa la clase padre de InputEmbeddings
        self.d_model = d_model # se le asigna el valor de d_model a la clase InputEmbeddings
        self.vocab_size = vocab_size # se le asigna el valor de vocab_size a la clase InputEmbeddings
        self.embedding = nn.Embedding(vocab_size, d_model) # se crea una capa de embedding con vocab_size entradas y d_model salidas 

    def forward(self, x): # funcion forward de la clase InputEmbeddings para insertar la posicion de las palabras
        return self.embedding(x) * math.sqrt(self.d_model) # se retorna el valor de la capa de embedding multiplicado por la raiz cuadrada de d_model
    

### Explicación de la clase PositionalEncoding
Esta clase se utiliza para incorporar informacion posicional en las entradas de un modelo, en pocas palabras nos ayuda a codificar la posicion de palabras en una secuencia para que el modelo pueda tener en cuenta el orden de estas, algo teorico que podemos ver aqui es cuando se crean matrices de un tamaño en especifico ya que lo que vimos es que la longitud de secuencia por el numero de dimenciones del modelo es donde se almacenan las codificaciones que nos dan por lo cual es super interesante.

In [7]:
# Clase para codificar la posicion de las palabras en un modelo en la red neuronal
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None: #Constructor de la clase PositionalEncoding
        super().__init__() #Inicializa la clase padre de PositionalEncoding
        self.d_model = d_model # se le asigna el valor de d_model a la clase PositionalEncoding en pocas palabras es el numero de dimensiones del modelo
        self.seq_len = seq_len # se le asigna el valor de seq_len a la clase PositionalEncoding en pocas palabras es el numero de palabras o la longitud en la secuencia
        self.dropout = nn.Dropout(dropout) # se crea una capa de dropout con el valor de dropout asignado a la clase PositionalEncoding
        
        pe = torch.zeros(seq_len, d_model) # se crea un tensor de ceros con las dimensiones de seq_len y d_model o matriz de ceros de seq_len x d_model
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # se crea un tensor con los valores de 0 a seq_len en float y se le agrega una dimension
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # se crea un tensor con los valores de 0 a d_model en float y se le agrega una dimension
        pe[:, 0::2] = torch.sin(position * div_term) # se asigna a la matriz pe en las columnas pares los valores de la funcion seno
        pe[:, 1::2] = torch.cos(position * div_term)  # se asigna a la matriz pe en las columnas impares los valores de la funcion coseno
        pe = pe.unsqueeze(0)  # se le agrega una dimension a la matriz pe para que quede de 1 x seq_len x d_model 
        self.register_buffer('pe', pe) # se registra el tensor pe en la clase PositionalEncoding como buffer 

    def forward(self, x): # funcion forward de la clase PositionalEncoding para codificar la posicion de las palabras
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model) se le suma a x el valor de pe en las dimensiones de x
        return self.dropout(x) # se retorna el valor de x despues de aplicarle el dropout

### Explicación de la clase ResidualConnection
Como su nombre lo dice implementa una conexion residual por lo cual estas nos ayudan como a mejorar el flujo de la informacion a traves de una red algo interesante de esta que podemos ver es que posee una capa de normalizacion la cual como vimos anteriormente esta ayuda a que la media y la desviacion esten como en equilibrio recordando que la media debe estar en 0 y la desviacion en 1, esto viendo en lo teorico de la clase esto pues ayuda a la convergencia como del entrenamiento del modelo.

In [8]:
# Clase para la conexion residual en un modelo de las redes neuronales
class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None: #Constructor de la clase ResidualConnection
            super().__init__() #Inicializa la clase padre de ResidualConnection
            self.dropout = nn.Dropout(dropout) # se crea una capa de dropout con el valor de dropout asignado a la clase ResidualConnection
            self.norm = LayerNormalization(features) # se crea una capa de normalizacion con el valor de features asignado a la clase ResidualConnection
    
        def forward(self, x, sublayer): # funcion forward de la clase ResidualConnection
            return x + self.dropout(sublayer(self.norm(x))) # se retorna el valor de x mas el valor de la capa de dropout aplicada a la capa de normalizacion de x


### Explicación de la clase MultiHeadAttentionBlock
Esta clase implementa un bloque con atencion multicabeza que es como una parte fundamentar de los transformadores, esta permite que el modelo pueda tener la atencion en diferentes partes de la entrada de manera simultanea como que mejorando la capacidad de capturar patrones complejos viendo la parte teorica por ejemplo al dividir en tareas o diferentes cabezas el modelo puede enfocarse en diferentes partes de la secuencia en pocas palabras en forma paralela.

In [9]:
# Clase para agregar un bloque de atencion multicabeza en un modelo de las redes neuronales transformadores
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None: #Constructor de la clase MultiHeadAttentionBlock
        super().__init__() #Inicializa la clase padre de MultiHeadAttentionBlock
        self.d_model = d_model  # se le asigna el valor de d_model a la clase MultiHeadAttentionBlock en pocas palabras es el numero de dimensiones del modelo
        self.h = h # se le asigna el valor de h a la clase MultiHeadAttentionBlock en pocas palabras es el numero de cabezas
        assert d_model % h == 0, "d_model is not divisible by h" # se verifica que d_model sea divisible por h

        self.d_k = d_model // h  # se le asigna el valor de d_k a la clase MultiHeadAttentionBlock en pocas palabras es el numero de dimensiones de la cabeza
        self.w_q = nn.Linear(d_model, d_model, bias=False) # se crea una capa lineal con d_model entradas y d_model salidas
        self.w_k = nn.Linear(d_model, d_model, bias=False) # se crea una capa lineal con d_model entradas y d_model salidas 
        self.w_v = nn.Linear(d_model, d_model, bias=False) # se crea una capa lineal con d_model entradas y d_model salidas
        self.w_o = nn.Linear(d_model, d_model, bias=False) # se crea una capa lineal con d_model entradas y d_model salidas
        self.dropout = nn.Dropout(dropout) # se crea una capa de dropout con el valor de dropout asignado a la clase MultiHeadAttentionBlock

    @staticmethod # metodo estatico para la atencion multicabeza en un modelo de las redes neuronales transformadores
    def attention(query, key, value, mask, dropout: nn.Dropout): # funcion de atencion multicabeza en un modelo de las redes neuronales transformadores
        d_k = query.shape[-1] # se le asigna el valor de d_k a la variable d_k
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k) # se calcula el producto punto de query y key y se divide por la raiz cuadrada de d_k
        if mask is not None: # si mask es diferente de None
            attention_scores.masked_fill_(mask == 0, -1e9) # se llena la matriz attention_scores con -1e9 en las posiciones donde mask sea igual a 0
        attention_scores = attention_scores.softmax(dim=-1)  # se aplica la funcion softmax a attention_scores en la ultima dimension
        if dropout is not None: # si dropout es diferente de None
            attention_scores = dropout(attention_scores) # se aplica el dropout a attention_scores
        return (attention_scores @ value), attention_scores # se retorna el producto punto de attention_scores y value y attention_scores

    def forward(self, q, k, v, mask): # funcion forward de la clase MultiHeadAttentionBlock
        query = self.w_q(q)  # se le asigna el valor de w_q a query
        key = self.w_k(k)  # se le asigna el valor de w_k a key 
        value = self.w_v(v) # se le asigna el valor de w_v a value

        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2) # se le da la forma a query para que quede de (batch, h, seq_len, d_k)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2) # se le da la forma a key para que quede de (batch, h, seq_len, d_k)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2) # se le da la forma a value para que quede de (batch, h, seq_len, d_k)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout) # se le asigna el valor de la funcion de atencion a x y self.attention_scores
        
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k) # se le da la forma a x para que quede de (batch, seq_len, d_model)

        return self.w_o(x) # se retorna el valor de w_o aplicado a x

### Explicación de la clase EncoderBlock
Lo que hace esta clase es como implementar un bloque de codificacion en un modelo de redes el cual nos ayuda en la codificacion de secuencias ya que nos permite procesar la entrada de manera paralela como se habia mencionado anteriormente es como un complemento de la clase anterior ya que es una parte clave en la ejecucion paralela y siempre prestando atencion a todas las partes de la secuencia, algo visto en teoria que se aplica en esta clase es la autoatencion ya que esta es o hace que un modelo preste atencion a diferentes partes de la secuencia de entrada sin importar su distancia entre estos.

In [10]:
# Clase para agregar un bloque de codificacion en un modelo de las redes neuronales transformadores
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None: #Constructor de la clase EncoderBlock
        super().__init__() #Inicializa la clase padre de EncoderBlock
        self.self_attention_block = self_attention_block # se le asigna el valor de self_attention_block a la clase EncoderBlock
        self.feed_forward_block = feed_forward_block # se le asigna el valor de feed_forward_block a la clase EncoderBlock
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)]) # se crea una lista de conexiones residuales con 2 elementos 

    def forward(self, x, src_mask): # funcion forward de la clase EncoderBlock
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask)) # se le asigna el valor de la primera conexion residual a x aplicando la atencion multicabeza en x con x y x y src_mask 
        x = self.residual_connections[1](x, self.feed_forward_block) # se le asigna el valor de la segunda conexion residual a x aplicando el feed forward block en x 
        return x # retorna el valor de x 
    

### Explicación de la clase Encoder
Esta clase es importante ya que toma una secuencia de entrada y la transforma como en una representacion digamos de alto nivel obviamente aplicando multiples capas de codificacion, entonces estas capas hacen que el modelo capture relaciones mas complejas dentro de una secuencia, tambien podemos observar que posee una capa de normalizacion lo cual ya habiamos hablado de ella que ayuda como a equilibrar la parte media y la desviacion como bien se vio en la parte teorica.

In [11]:
# Clase para implementar el encoder en un modelo de las redes neuronales transformadores
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None: #Constructor de la clase Encoder
        super().__init__() #Inicializa la clase padre de Encoder
        self.layers = layers # se le asigna el valor de layers a la clase Encoder en pocas palabras es el numero de capas
        self.norm = LayerNormalization(features) # se crea una capa de normalizacion con el valor de features asignado a la clase Encoder 

    def forward(self, x, mask): # funcion forward de la clase Encoder
        for layer in self.layers: # para cada capa en layers se le asigna el valor de x aplicando la capa y mask 
            x = layer(x, mask) # se le asigna el valor de x aplicando la capa y mask 
        return self.norm(x) # retorna el valor de x aplicando la capa de normalizacion


### Explicación de la clase DecoderBlock
Este implementa un bloque de decodificacion como su nombre lo dice en un modelo de redes, el cual esta encargado de procesar la salida generada por el codificador, el objetivo teorico importante son las caracteristicas que se le asignan, ya que este como numero de caracteristicas o digamos dimensiones de modelo identico al codificador pues este define el tamaño de los vectores de representacion que genera el decodificador.

In [12]:
# Clase para agregar un bloque de decodificacion en un modelo de las redes neuronales transformadores
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None: #Constructor de la clase DecoderBlock
        super().__init__() #Inicializa la clase padre de DecoderBlock
        self.self_attention_block = self_attention_block # se le asigna el valor de self_attention_block a la clase DecoderBlock
        self.cross_attention_block = cross_attention_block # se le asigna el valor de cross_attention_block a la clase DecoderBlock 
        self.feed_forward_block = feed_forward_block # se le asigna el valor de feed_forward_block a la clase DecoderBlock 
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)]) # se crea una lista de conexiones residuales con 3 elementos 

    def forward(self, x, encoder_output, src_mask, tgt_mask): # funcion forward de la clase DecoderBlock
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask)) # se le asigna el valor de la primera conexion residual a x aplicando la atencion multicabeza en x con x y x y tgt_mask
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask)) # se le asigna el valor de la segunda conexion residual a x aplicando la atencion multicabeza en x con encoder_output y encoder_output y src_mask
        x = self.residual_connections[2](x, self.feed_forward_block) # se le asigna el valor de la tercera conexion residual a x aplicando el feed forward block en x
        return x # retorna el valor de x
    

### Explicación de la clase Decoder
Esta clase lo que hace es implementar el decoder en un modelo de red neuronal, el cual toma tanto la secuencia que se genera como la secuencia objetiva generada el objetivo teorico de esta es que posee tipo entrada, proceso y salida como en los programas, en la entrada tenemos x, encoder_output, src_mask, tgt_mask, en el proceso solo va recorriendo la capas del decodificador para aplicar la atencion propia a esa capa y en la salida vemos que al igual que clases anteriores hay una capa de normalizacion igual como se habia explicado anteriormente sobre esta.

In [13]:
# Clase para implementar el decoder en un modelo de las redes neuronales transformadores
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None: #Constructor de la clase Decoder
        super().__init__() #Inicializa la clase padre de Decoder
        self.layers = layers # se le asigna el valor de layers a la clase Decoder en pocas palabras es el numero de capas
        self.norm = LayerNormalization(features) # se crea una capa de normalizacion con el valor de features asignado a la clase Decoder

    def forward(self, x, encoder_output, src_mask, tgt_mask): # funcion forward de la clase Decoder
        for layer in self.layers: # para cada capa en layers se le asigna el valor de x aplicando la capa y encoder_output, src_mask y tgt_mask
            x = layer(x, encoder_output, src_mask, tgt_mask) # se le asigna el valor de x aplicando la capa y encoder_output, src_mask y tgt_mask
        return self.norm(x) # retorna el valor de x aplicando la capa de normalizacion


### Explicación de la clase ProjectionLayer
Como el nombre lo dice implementa una capa de proyeccion en un modelo esto como para mapear las salidas de una red hacia un espacio x digamos o podria ser una categoria, esta al igual que la anterior contiene entrada, proceso y salida el cual con el objetivo teorico es que si se aplicara una transformacion lineal esto convertiria cada vector de un tamaño en especifico un un vector de tamaño como dice la clase un tamño vocab_size lo cual esto ayuda a que se realice una prediccion sobre la posicion de esta secuencia en especifico.

In [14]:
# Clase para agregar en un modelo la proyeccion de las capas de las redes neuronales
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None: #Constructor de la clase ProjectionLayer
        super().__init__() #Inicializa la clase padre de ProjectionLayer
        self.proj = nn.Linear(d_model, vocab_size) # se crea una capa lineal con d_model entradas y vocab_size salidas

    def forward(self, x) -> None: # funcion forward de la clase ProjectionLayer
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size) 
        return self.proj(x) # retorna el valor de x aplicando la capa lineal
    

### Explicación de la clase Transformer
Aqui viene la cereza del pastel, esta clase implementa el modelo completo de red neuronal transformers, el cual como podemos ver en la clase esta incluye fases como la codificacion y decodificacion en conjunto con un proceso de proyeccion de salidas, algo interesante que me di cuenta es que aqui aplica como las partes de las clase anteriores de encode, decode y project ya que el objetivo teorico para esta es que ambas como funciones son las mismas que las que vimos anteriomente, el cual cada una de ellas posee entradas proceso y salida entonces para esta clase tenemos 3 entradas, 3 proceso y 3 salidas interesante no? ahora entiendo porque es la cereza del pastel esta clase.

In [15]:
# Clase para implementar el modelo de las redes neuronales transformadores
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None: #Constructor de la clase Transformer
        super().__init__() #Inicializa la clase padre de Transformer
        self.encoder = encoder # se le asigna el valor de encoder a la clase Transformer
        self.decoder = decoder # se le asigna el valor de decoder a la clase Transformer
        self.src_embed = src_embed # se le asigna el valor de src_embed a la clase Transformer
        self.tgt_embed = tgt_embed # se le asigna el valor de tgt_embed a la clase Transformer
        self.src_pos = src_pos # se le asigna el valor de src_pos a la clase Transformer
        self.tgt_pos = tgt_pos # se le asigna el valor de tgt_pos a la clase Transformer
        self.projection_layer = projection_layer # se le asigna el valor de projection_layer a la clase Transformer

    def encode(self, src, src_mask): # funcion para codificar en un modelo de las redes neuronales transformadores
        src = self.src_embed(src) # se le asigna el valor de src_embed a src
        src = self.src_pos(src) # se le asigna el valor de src_pos a src
        return self.encoder(src, src_mask) # retorna el valor de src aplicando el encoder y src_mask
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor): # funcion para decodificar en un modelo de las redes neuronales transformadores
        tgt = self.tgt_embed(tgt) # se le asigna el valor de tgt_embed a tgt
        tgt = self.tgt_pos(tgt) # se le asigna el valor de tgt_pos a tgt
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask) # retorna el valor de tgt aplicando el decoder y encoder_output, src_mask y tgt_mask
    
    def project(self, x): # funcion para proyectar en un modelo de las redes neuronales transformadores
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x) # retorna el valor de x aplicando la capa de proyeccion
    

### Explicación de la funcion build_transformer
Esta funcion crea un modelo transformer personalizado para tareas de procesamiento de secuencias, como vimos anteriormente la traduccion de texto o generacion de secuencias, al ver esta funcion nos damos cuenta que llama todas las clases es super genial como manda a llamar todas las clases como el decodificador, codificador las proyecciones, etc. lo cual es impresionante, ahora para el objetivo vemos que esta posee parametros como por ejemplo el tamaño del vocabulario de la entrada, tambien el tamaño de la salida, la longitud de la secuencia de entrada y de la del destino, las dimensiones, el numero de capas y asi sucesivamente ahora vemos porque esta completo este modelo de transformers.

In [16]:
# Funcion para construir un modelo de las redes neuronales transformadores
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer: # funcion para construir un modelo de las redes neuronales transformadores
    src_embed = InputEmbeddings(d_model, src_vocab_size) # se crea una capa de embedding con d_model entradas y src_vocab_size salidas
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size) # se crea una capa de embedding con d_model entradas y tgt_vocab_size salidas

    src_pos = PositionalEncoding(d_model, src_seq_len, dropout) # se crea una capa de codificacion de posicion con d_model, src_seq_len y dropout
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout) # se crea una capa de codificacion de posicion con d_model, tgt_seq_len y dropout
    
    encoder_blocks = [] # se crea una lista de bloques de codificacion
    for _ in range(N): # para cada capa en N se crea un bloque de codificacion
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # se crea un bloque de atencion multicabeza con d_model, h y dropout
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # se crea un bloque de feed forward con d_model, d_ff y dropout
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout) # se crea un bloque de codificacion con d_model, encoder_self_attention_block, feed_forward_block y dropout
        encoder_blocks.append(encoder_block) # se agrega el bloque de codificacion a la lista de bloques de codificacion

    decoder_blocks = [] # se crea una lista de bloques de decodificacion 
    for _ in range(N): # para cada capa en N se crea un bloque de decodificacion
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # se crea un bloque de atencion multicabeza con d_model, h y dropout
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # se crea un bloque de atencion multicabeza con d_model, h y dropout
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # se crea un bloque de feed forward con d_model, d_ff y dropout
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout) # se crea un bloque de decodificacion con d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block y dropout
        decoder_blocks.append(decoder_block) # se agrega el bloque de decodificacion a la lista de bloques de decodificacion
    
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks)) # se crea un encoder con d_model y nn.ModuleList(encoder_blocks)
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks)) # se crea un decoder con d_model y nn.ModuleList(decoder_blocks)
    
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size) # se crea una capa de proyeccion con d_model y tgt_vocab_size
    
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer) # se crea un modelo de las redes neuronales transformadores con encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos y projection_layer

    for p in transformer.parameters(): # para cada parametro en transformer se inicializa con xavier_uniform
        if p.dim() > 1: # si la dimension de p es mayor a 1
            nn.init.xavier_uniform_(p) # se inicializa p con xavier_uniform
    
    return transformer # retorna el modelo de las redes neuronales transformadores

## Referencias
1. Merritt, R. (2022, 19 abril). ¿Qué es un modelo transformer? | Blog de NVIDIA. Blog Oficial de NVIDIA Latino América. https://la.blogs.nvidia.com/blog/que-es-un-modelo-transformer/
2. Giacaglia, G. (2024, 23 marzo). How Transformers Work - towards data science. Medium. https://towardsdatascience.com/transformers-141e32e69591
3. Loss, A. (2024, 30 enero). From Neural Networks to Transformers: The Evolution of Machine Learning. DATAVERSITY. https://www.dataversity.net/from-neural-networks-to-transformers-the-evolution-of-machine-learning/
4. Jacon, T., & Jacon, T. (2023, 20 diciembre). Codificador y decodificador de IA en el procesamiento del lenguaje natural - Planeta Chatbot. Planeta Chatbot - Comunidad de expertos en IA Conversacional. https://planetachatbot.com/codificador-y-decodificador-de-ia-en-el-procesamiento-del-lenguaje-natural/
5. Qué son los Transformers en PLN: ventajas e inconvenientes. (s. f.). https://blog.pangeanic.com/es/que-son-los-transformers-en-pln
    